In [ ]:
import pandas as pd

image_list = pd.read_csv("data/raw_image_list.csv")
image_list.head()

In [ ]:
from pathlib import Path
import shutil

remote = (Path("/allen") / image_list.loc[1]["Isilon path"][1:] / image_list.loc[1]["File Name"]).resolve(strict=True)
print(f"Remote filepath: {remote}, Size in GB: {remote.stat().st_size / 1000 / 1000 / 1000}")

# local = Path(f"/tmp/{czi.name}")
# print(f"Beginning copy to: {local}")
# shutil.copy(str(czi), str(local))
# local.resolve(strict=True)
# print(f"Copy complete, located at: {local}")

In [ ]:
from timelapse_tools.movies import generate_movie

generate_movie(remote, "/Users/jacksonb/Desktop/test.mp4", overwrite=True, series_range=slice(0, 100, 4), show_progress=True, C=0, fps=4)